In [49]:
from google.colab import files
uploaded = files.upload()

Saving Bengaluru_House_Data.csv to Bengaluru_House_Data (1).csv


In [50]:
import pandas as pd

df = pd.read_csv("Bengaluru_House_Data.csv")
display(df.head())

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [51]:
display(df.tail())

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,ArsiaEx,3453,4.0,0.0,231.0
13316,Super built-up Area,Ready To Move,Richards Town,4 BHK,NaN,3600,5.0,NaN,400.0
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,Mahla T,1141,2.0,1.0,60.0
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,SollyCl,4689,4.0,1.0,488.0
13319,Super built-up Area,Ready To Move,Doddathoguru,1 BHK,NaN,550,1.0,1.0,17.0


In [52]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


None

# **Cleaning : **

In [53]:
#Detecting the NaN values
missing = df.isnull().sum()
print("Missing values per column:")
print(missing)

Missing values per column:
area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64


In [54]:
#1. Drop Row with all Entries as NaN
df = df.dropna(how='all')
print('Shape after dropping all-missing rows:', df.shape)

Shape after dropping all-missing rows: (13320, 9)


In [55]:
#2. Statistical Fill
bath_median = df['bath'].median()
balcony_median = df['balcony'].median()
location_mode = df['location'].mode()[0]

df_stat= df.fillna({'bath': bath_median, 'balcony': balcony_median, 'location': location_mode})

print("Missing values after statistical fill:")
print(df_stat.isnull().sum())

Missing values after statistical fill:
area_type          0
availability       0
location           0
size              16
society         5502
total_sqft         0
bath               0
balcony            0
price              0
dtype: int64


# **Handling NaN : **

In [56]:
print(df.isnull().sum())

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64


In [57]:
#ForwardFill
df1 = df.fillna(method='ffill')
print("Remaining missing values after forward fill:")
print(df1.isnull().sum())

Remaining missing values after forward fill:
area_type       0
availability    0
location        0
size            0
society         0
total_sqft      0
bath            0
balcony         0
price           0
dtype: int64


/tmp/ipython-input-57-3604591540.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df1 = df.fillna(method='ffill')


In [58]:
#ForwardFill then BackwardFill
df2 = df.copy()
df2['bath'] = df2['bath'].bfill()
df2['bath'] = df2['bath'].ffill()

df['bath'] = df['bath'].bfill()
df['bath'] = df['bath'].ffill()

df['balcony'] = df['balcony'].bfill()
df['balcony'] = df['balcony'].ffill()

df['society'] = df['society'].bfill()
df['society'] = df['society'].ffill()

print("Missing values after both fills:", df2['bath'].isnull().sum())

Missing values after both fills: 0


# **Filtering :**

In [59]:
#Based on House Price Above 50L :
df_pricing = df[df['price'] < 50]
print("Houses under ₹50 lakhs:", df_pricing.shape[0])

Houses under ₹50 lakhs: 3279


In [60]:
# Cheap and Costly Properties
cheap_costly = df.query('price < 30 or price > 100')
print("Total cheap/premium properties:", len(cheap_costly))
print(cheap_costly.head())

Total cheap/premium properties: 4650
               area_type   availability          location       size  society  \
1             Plot  Area  Ready To Move  Chikka Tirupathi  4 Bedroom  Theanmp   
6   Super built-up  Area         18-May  Old Airport Road      4 BHK  Jaades    
7   Super built-up  Area  Ready To Move      Rajaji Nagar      4 BHK  Brway G   
9             Plot  Area  Ready To Move      Gandhi Bazar  6 Bedroom  Prrry M   
11            Plot  Area  Ready To Move        Whitefield  4 Bedroom  Prrry M   

   total_sqft  bath  balcony  price  
1        2600   5.0      3.0  120.0  
6        2732   4.0      1.0  204.0  
7        3300   4.0      1.0  600.0  
9        1020   6.0      2.0  370.0  
11       2785   5.0      3.0  295.0  


# **Sorting : **

In [61]:
#Single-Column Sort on Price
df_sorted_price = df.sort_values(by='price', ascending=False)
print("Top 5 Most Expensive Properties : ")
print(df_sorted_price[['location', 'price']].head(5))

Top 5 Most Expensive Properties : 
              location   price
13067   Defence Colony  3600.0
11080      Ashok Nagar  2912.0
13200   Defence Colony  2800.0
11763  Sadashiva Nagar  2736.0
3180   Shanthala Nagar  2700.0


# **GroupBy : **

In [62]:
#Grouping Avg Property Price by Location
avg_price = df.groupby('location')['price'].mean().sort_values(ascending=False)
print("Average price by location:")
print(avg_price.head())

Average price by location:
location
Cubbon Road       1900.000000
Ashok Nagar       1486.000000
Defence Colony    1167.714286
Yemlur            1093.388889
Church Street     1068.000000
Name: price, dtype: float64


# **Creating Column : **

In [68]:
#Ratio or Bath to Balcony to check the Luxury of the House
df['bath_balcony_ratio'] = (df['bath'] / df['balcony']).round(2)
df['bath_balcony_ratio'] = df['bath_balcony_ratio'].ffill()

In [64]:
#Sets the Type of Property Based on Area Type
df['property_type'] = df['area_type'].map({
    'Plot  Area': 'Plot',
    'Built-up  Area': 'Flat+',
    'Super built-up  Area': 'Flat++',
    'Carpet  Area': 'Flat'
})

In [69]:
df.to_csv("Cleaned_Bengaluru_House_Data.csv", index=False)

from google.colab import files
files.download("Cleaned_Bengaluru_House_Data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>